## Ejemplo 2: ClassicModels

Trigger en PostgreSQL que verifica que la suma de `(quantityOrdered * priceEach)` de todos los pedidos de un cliente no supere su límite de crédito (`creditLimit`) más la suma de sus pagos (`payments.amount`). Este trigger previene que se realicen operaciones `INSERT` o `UPDATE` en la tabla `orderDetails` si se viola esta condición.

### Código del Trigger y la Función

```sql
-- Crear función para manejar el trigger
CREATE OR REPLACE FUNCTION check_customer_credit_limit()
RETURNS TRIGGER AS $$
DECLARE
    total_order_value NUMERIC;
    total_payments NUMERIC;
    available_credit NUMERIC;
BEGIN
    -- Calcular el total del pedido actual
    total_order_value := (
        SELECT SUM(quantityOrdered * priceEach)
        FROM orderDetails
        WHERE orderNumber IN (
            SELECT orderNumber
            FROM orders
            WHERE customerNumber = (
                SELECT customerNumber
                FROM orders
                WHERE orderNumber = COALESCE(NEW.orderNumber, OLD.orderNumber)
                LIMIT 1
            )
        )
    );

    -- Calcular el total de pagos del cliente
    total_payments := (
        SELECT COALESCE(SUM(amount), 0)
        FROM payments
        WHERE customerNumber = (
            SELECT customerNumber
            FROM orders
            WHERE orderNumber = COALESCE(NEW.orderNumber, OLD.orderNumber)
            LIMIT 1
        )
    );

    -- Calcular el crédito disponible del cliente
    available_credit := (
        SELECT creditLimit
        FROM customers
        WHERE customerNumber = (
            SELECT customerNumber
            FROM orders
            WHERE orderNumber = COALESCE(NEW.orderNumber, OLD.orderNumber)
            LIMIT 1
        )
    ) + total_payments;

    -- Verificar que el total del pedido no exceda el crédito disponible
    IF total_order_value > available_credit THEN
        RAISE EXCEPTION 'Pedido no permitido: El total de pedidos excede el crédito disponible (%), Cliente: %', 
            available_credit, 
            (SELECT customerNumber FROM orders WHERE orderNumber = COALESCE(NEW.orderNumber, OLD.orderNumber) LIMIT 1);
    END IF;

    RETURN NULL; -- Los triggers AFTER no requieren devolver nada
END;
$$ LANGUAGE plpgsql;

-- Crear el trigger en la tabla orderDetails
CREATE TRIGGER trg_check_customer_credit_limit
BEFORE INSERT OR UPDATE ON orderDetails
FOR EACH ROW
EXECUTE FUNCTION check_customer_credit_limit();
```

### Explicación del Código

1. **Función `check_customer_credit_limit`:**
   - Calcula el total de pedidos realizados por el cliente (`total_order_value`).
   - Calcula los pagos acumulados realizados por el cliente (`total_payments`).
   - Suma el límite de crédito del cliente (`creditLimit`) con sus pagos para calcular el crédito disponible (`available_credit`).
   - Verifica si el valor total de los pedidos excede el crédito disponible. Si es así, lanza un error con `RAISE EXCEPTION`.

2. **Trigger `trg_check_customer_credit_limit`:**
   - Se activa antes de las operaciones `INSERT` y `UPDATE` en la tabla `orderDetails`.
   - Llama a la función `check_customer_credit_limit` para validar que el pedido cumpla con las restricciones.

### Ejemplo de Uso

1. **Insertar un pedido válido:**
   ```sql
   INSERT INTO orderDetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
   VALUES (10103, 'S10_1678', 5, 50.00, 1); -- Si es válido, se inserta
   ```

2. **Insertar un pedido inválido:**
   ```sql
   INSERT INTO orderDetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
   VALUES (10103, 'S10_1678', 1000, 50.00, 1); -- Lanza error si excede el crédito disponible
   ```

3. **Actualizar un pedido válido:**
   ```sql
   UPDATE orderDetails
   SET quantityOrdered = 10, priceEach = 60.00
   WHERE orderNumber = 10103 AND productCode = 'S10_1678';
   ```

4. **Actualizar un pedido inválido:**
   ```sql
   UPDATE orderDetails
   SET quantityOrdered = 1000, priceEach = 60.00
   WHERE orderNumber = 10103 AND productCode = 'S10_1678'; -- Lanza error si excede el crédito disponible
   ```

Este código asegura que los pedidos en `orderDetails` respeten el límite de crédito combinado con los pagos del cliente, preservando la integridad de las reglas de negocio.